# Answers
1. (3)
2. (2) https://membres-ljk.imag.fr/Roland.Hildebrand/trier_vortrag.pdf
3. (2)
4. (4)

In [50]:
## ####################################################
import numpy as np
from sklearn.datasets import load_breast_cancer

# load the data
X, y = load_breast_cancer(return_X_y=True)  ## X input, y output
print(X.shape, y.shape)
## to convert the {0,1} output into {-1,+1}
y = 2 * y -1

mdata,ndim=X.shape                                   ## size of the data 

iscale = 1   ## =0 no scaling, =1 scaling the by the maximum absolute value
if iscale == 1: 
  X /= np.outer(np.ones(mdata),np.max(np.abs(X),0))

niter = 10 ## number of iteration 


(569, 30) (569,)


In [51]:
## initialize eta, lambda for the primal algorithm
eta=0.1              ##  step size
xlambda=0.01          ## balancing constant between loss and regularization

w1 = np.zeros(ndim)
for i in range(niter):
  X_i = X[i,:]
  y_i = y[i]
  upDir = y_i*np.dot(w1,X_i)
  if upDir < 1:
    J = -(y_i*X_i)+(xlambda*w1)
  else:
    J = xlambda*w1
  w1 -= eta*J

In [52]:
## set the penalty constant for the dual algorithm
C = 1000

alpha = np.zeros(ndim)
for i in range(niter):
  X_i = X[i,:]
  denominator = np.dot(X_i,X_i)
  nominator = 1
  for j in range(niter):
    if(i==j):
      pass
    X_j = X[j,:]
    linKernel = np.dot(X_i,X_j)
    nominator -= y[i]*alpha[j]*y[j] * linKernel
    #print(f'y_{i} = {y[i]}, alpha_{j}={alpha[j]}, y_{j}={y[j]}')
    #print(f'For iteration i={i}, j={j} the nominator is {nominator}')
  alpha[i] = nominator/denominator
  #print(f'Pre penalty alpha_{i}: {alpha[i]}')
  alpha[i] = min(C/mdata, max(0, alpha[i]))
  #print(f'Post penalty alpha_{i}: {alpha[i]}')

  
w2 = sum(alpha[i]*y[i]*X[i,:] for i in range(ndim))
#w2 = 0
#for (i, a_i) in enumerate(alpha):
#  w2 += a_i*y[i]*X[i,:]

In [63]:
def stocastic_dual_coordinate_ascent(X,y, x_i, y_i):
    C = 1000
    alpha = np.zeros(ndim)
    for j in range(niter):
        y_j= y[j]
        x_j = X[j,:]
        sum = alpha[j] * y_j * np.dot(x_i, x_j)
    alpha_i = (1 - y_i * sum) / (np.dot(x_i,x_i))
    alpha_i = min(C/mdata, max(alpha_i, 0))
    return alpha_i

def dual_soft_margin(X,y):
    weights = np.zeros(ndim)
    for i in range(niter):
        y_i = y[i]
        x_i = X[i,:]
        alpha_i= stocastic_dual_coordinate_ascent(X,y,x_i,y_i)
        weights = weights + alpha_i*y_i*x_i
    return weights
  
#w2 = dual_soft_margin(X, y) 

TypeError: 'builtin_function_or_method' object does not support item assignment

In [59]:
from math import ceil

ceil(np.corrcoef(w1,w2)[1,0]*100)/100

1.0